**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [75]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd,adam
from keras.layers import Flatten,Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The $\epsilon$ value is the probability to play a random action at each state. This is called exploration : even when the agent managed to have a good score, he will continue to try to improve its performance by testing other actions. This is very important for several reasons :
- The optimal policy may include some suboptimal actions: take for example a trajectory that gives rewards -1,-1,-1,-1,-1,10000 vs a strategy that gives 1,1,1,1,1,1. With a gradient update at each step and no exploration a DQN agent will choose the second one even it is not the best. It is quite similar as being stuck in a local optimal in a gradient descent
- The initial distribution of the Q-value is random and the best actions to select may have very low probability

Choosing the right  epsilon is hard. Too low and the model may be stuck in a local optimum. Too hight value will decrease the agent performance as it will continue to play random move even after convergence. 

Finally note that there are alternative solution to epsilon exploration. For instance actor critic model predict probability for each action and draw an action based on that probablity. 



***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Position encodes the position where the agent can go and board the cheese pieces positions

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(3)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):


        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
                  .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:

# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)
# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 2.5/9.0. Average score (-6.5)
Win/lose count 5.5/10.0. Average score (-5.5)
Win/lose count 1.5/5.0. Average score (-4.833333333333333)
Win/lose count 6.5/12.0. Average score (-5.0)
Win/lose count 9.0/13.0. Average score (-4.8)
Win/lose count 5.0/8.0. Average score (-4.5)
Win/lose count 8.5/5.0. Average score (-3.357142857142857)
Win/lose count 5.5/8.0. Average score (-3.25)
Win/lose count 7.5/6.0. Average score (-2.7222222222222223)
Win/lose count 2.5/8.0. Average score (-3.0)
Final score: -3.0


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




Using Bellman equation we have : 
\begin{align}
Q^{\pi}(s,a)&=E[ \sum_{k=0}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
& = E[ r(s,a) + \sum_{k=1}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
& =  r(s,a) + \gamma E[E[\sum_{k=0}^{\infty}[\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']|s_t = s, a_t = a] \\
& =  r(s,a) + \gamma \sum_{(s',a')}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )[E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
&=  \sum_{(s',a')} (p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) r(s,a) + \gamma \sum_{(s',a')}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align}
We deduce that the optimal policy is
\begin{align}
Q^{*}(s,a) & = \max_{\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\
&= \max_{\pi}r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi}(s',a')\\
&= r(s,a) + \gamma \max_{\pi} \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') \qquad \text{with } \pi=(a',\pi')
\end{align}
As we have :
\begin{equation*}
\max_{\pi} \sum_{(s')} p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') \leq \sum_{(s',a')} p(s_{t+1}=s' |s_t = s, a_t = a ) \max_{\pi}Q^{\pi'}(s',a') 
\end{equation*}
we can intervert the sums.
Then we have considering $\hat{\pi}(s',a') = \arg\max_{\pi'} Q^{\pi'}(s',a')$ 
\begin{align} \sum{s'} p(s{t+1}=s'|s_t = s, at = a ) \max{\pi'}Q^{\pi'}(s',a') &= \sum{s'} p(s{t+1}=s' |s_t = s, at = a ) Q^{\hat{\pi}(s',a')}(s',a') \\ &\leq \max_{\pi'} \sum{s'} p(s{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') \end{align}

So finally : \begin{align} Q^{}(s,a)&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, at = a )\max_{a'}\max_{\pi'}Q^{\pi'}(s',a') \\ &= r(s,a) + \gamma \sum(p(s{t+1}=s' |s_t = s, at = a )\max_{a'} Q^{}(s',a') \\ &=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max{a'}Q^{*}(s',a')]. \end{align}

We propose the bjective function

\begin{equation*}
\mathcal{L}(\theta)=E_{s'*\sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
import random
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)>=self.max_memory:
            self.memory=self.memory[1:]
            self.memory.append(m)
        else :
            self.memory.append(m)

    def random_access(self):
        return random.choice(self.memory)
    def random_batch(self,batch_size):
        return np.random.choice(self.memory,batch_size)
        

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [122]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        s = s.reshape((1,5,5,self.n_state))
        
        values = self.model.predict(s)
        #print(values)
        return np.argmax(values)
        
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        #batch = self.memory.random_batch(self.batch_size)
        for i in range(self.batch_size):
            sample = self.memory.random_access()
            #s_,n_s_,a_,r_,game_over_ = batch[i][0],batch[i][1],batch[i][2],batch[i][3]
            s_,n_s_,a_,r_,game_over_ = sample[0],sample[1],sample[2],sample[3],sample[4]
            
            if game_over_:
                target_q[i,a_] = r_
            else:
                target = (r_ + self.discount * np.amax(self.model.predict(n_s_.reshape((1,5,5,self.n_state)))[0]))
                target_q[i,:] =  self.model.predict(s_.reshape((1,5,5,self.n_state)))[0]
                target_q[i,a_] = target
            input_states[i] = s_
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        # best model tested, see details below
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(50,activation ='relu')) 
        model.add(Dense(64,activation ='relu')) 
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse") 
        model.summary()
        self.model = model
        

In [97]:
global model_score 
model_score = {}
class DQN_generic(DQN):
    def __init__(self, model,*args, lr=0.1,**kwargs):
        super(DQN_generic, self).__init__( *args,**kwargs)
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.summary()
        self.model = model
def test_agent(agent,env,epochs=50,viz=True):
    score = 0
        
    for e in range(epochs):
        state = env.reset()
        game_over = False
        win = 0
        lose = 0

        while not game_over:
            action = agent.act(state)
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
           
        
        score = score + win-lose
    if viz:
        print('Final score: '+str(score/epochs))
    return score/epochs
def test_model(model,n_epochs=51,id_model=0,lr=.1,epsilon=0.1):
    agent = DQN_generic(model,size, lr=lr, epsilon=epsilon, memory_size=2000, batch_size = 32)
    train(agent, env, n_epochs, prefix='fc_train')
    model_score[id_model]=test_agent(agent,env)
    
    
        

In [84]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
#train(agent, env, epochs_train, prefix='fc_train')
train(agent, env, 51, prefix='fc_train')

HTML(display_videos('fc_train50.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_35 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_130 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_131 (Dense)            (None, 64)                3264      
_________________________________________________________________
dense_132 (Dense)            (None, 4)                 260       
Total params: 6,074
Trainable params: 6,074
Non-trainable params: 0
_________________________________________________________________
Epoch 000/051 | Loss 0.0060 | Win/lose count 3.0/5.0 (-2.0)
Epoch 001/051 | Loss 0.0832 | Win/lose count 4.0/6.0 (-2.0)
Epoch 002/051 | Loss 0.0116 | Win/lose count 5.5/5.0 (0.5)
Epoch 003/051 | Loss 0.0049 | Win/lose count 2.0/5.0 (-3.0)
Epoch 004/051 | Loss 0.0071 | Win

I tried several architecture :
- architecture 1 : one 30 fully connected layer
- architecture 2 : one 50 fully connected layer
- architecture 3 : one 50 fully connected layer, one 64 fully connected layer
- architecture 4 : one 50 fully connected layer, one 64 fully connected layer
- architecture 5 : one 50 fully connected layer, one 64 fully connected layer, one 32 fully connected layer
- architecture 6 : same as 5 with batch normalization between the layers.


In [70]:
model6= Sequential()
model6=Sequential()
model6.add(Flatten(input_shape=(5,5,2,)))
model6.add(Dense(50,activation="relu"))
model6.add(BatchNormalization())
model6.add(Dense(64,activation="relu"))
model6.add(BatchNormalization())
model6.add(Dense(32,activation="relu"))
model6.add(BatchNormalization())
model6.add(Dense(8,activation="relu"))
model6.add(BatchNormalization())
model6.add(Dense(4,activation="softmax"))
test_model(model6,n_epochs=51)
HTML(display_videos('fc_train50.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_33 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_123 (Dense)            (None, 50)                2550      
_________________________________________________________________
batch_normalization_1 (Batch (None, 50)                200       
_________________________________________________________________
dense_124 (Dense)            (None, 64)                3264      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
dense_125 (Dense)            (None, 32)                2080      
_________________________________________________________________
batch_normalization_3 (Batch (None, 32)                128       
__________

In [60]:
model5= Sequential()
model5=Sequential()
model5.add(Flatten(input_shape=(5,5,2,)))
model5.add(Dense(50,activation="relu"))
model5.add(Dense(64,activation="relu"))
model5.add(Dense(32,activation="relu"))
model5.add(Dense(8,activation="relu"))
model5.add(Dense(4,activation="softmax"))
test_model(model5,n_epochs=51,id_model=5)
HTML(display_videos('fc_train50.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_25 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_99 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_100 (Dense)            (None, 64)                3264      
_________________________________________________________________
dense_101 (Dense)            (None, 32)                2080      
_________________________________________________________________
dense_102 (Dense)            (None, 8)                 264       
_________________________________________________________________
dense_103 (Dense)            (None, 4)                 36        
Total params: 8,194
Trainable params: 8,194
Non-trainable params: 0
_________________________________________________________________
Epoch 00

In [62]:
model4=Sequential()
model4.add(Flatten(input_shape=(5,5,2,)))
model4.add(Dense(50,activation="relu"))
model4.add(Dense(64,activation="relu"))
model4.add(Dense(32,activation="relu"))
model4.add(Dense(4,activation="softmax"))
test_model(model4,n_epochs=51,id_model=4)
HTML(display_videos('fc_train50.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_27 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_108 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_109 (Dense)            (None, 64)                3264      
_________________________________________________________________
dense_110 (Dense)            (None, 32)                2080      
_________________________________________________________________
dense_111 (Dense)            (None, 4)                 132       
Total params: 8,026
Trainable params: 8,026
Non-trainable params: 0
_________________________________________________________________
Epoch 000/051 | Loss 0.0001 | Win/lose count 2.0/1.0 (1.0)
Epoch 001/051 | Loss 0.0024 | Win/lose count 3.0/2.0 (1.0)
Epoch 002/051 | Loss 0

In [63]:
model3=Sequential()
model3.add(Flatten(input_shape=(5,5,2,)))
model3.add(Dense(50,activation="relu"))
model3.add(Dense(64,activation="relu"))
model3.add(Dense(4,activation="softmax"))
test_model(model3,n_epochs=51,id_model=3)
HTML(display_videos('fc_train50.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_28 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_112 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_113 (Dense)            (None, 64)                3264      
_________________________________________________________________
dense_114 (Dense)            (None, 4)                 260       
Total params: 6,074
Trainable params: 6,074
Non-trainable params: 0
_________________________________________________________________
Epoch 000/051 | Loss 0.0065 | Win/lose count 3.0/4.0 (-1.0)
Epoch 001/051 | Loss 0.0103 | Win/lose count 3.5/7.0 (-3.5)
Epoch 002/051 | Loss 0.0003 | Win/lose count 1.0/7.0 (-6.0)
Epoch 003/051 | Loss 0.0103 | Win/lose count 5.0/6.0 (-1.0)
Epoch 004/051 | Loss 0.0325 | Wi

In [65]:
model2=Sequential()
model2.add(Flatten(input_shape=(5,5,2,)))
model2.add(Dense(50,activation="relu"))
model2.add(Dense(4,activation="softmax"))
test_model(model2,n_epochs=51,id_model=2)
HTML(display_videos('fc_train50.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_30 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_117 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_118 (Dense)            (None, 4)                 204       
Total params: 2,754
Trainable params: 2,754
Non-trainable params: 0
_________________________________________________________________
Epoch 000/051 | Loss 0.0007 | Win/lose count 2.5/3.0 (-0.5)
Epoch 001/051 | Loss 0.0080 | Win/lose count 1.0/1.0 (0.0)
Epoch 002/051 | Loss 0.0024 | Win/lose count 5.5/6.0 (-0.5)
Epoch 003/051 | Loss 0.0270 | Win/lose count 5.0/8.0 (-3.0)
Epoch 004/051 | Loss 0.0065 | Win/lose count 6.0/4.0 (2.0)
Epoch 005/051 | Loss 0.0088 | Win/lose count 5.0/6.0 (-1.0)
Epoch 006/051 | Loss 0.0210 | Win/lose count 3

In [72]:
model1=Sequential()
model1.add(Flatten(input_shape=(5,5,2,)))
model1.add(Dense(30,activation="relu"))
model1.add(Dense(4,activation="softmax"))
test_model(model1,n_epochs=51,id_model=1)
HTML(display_videos('fc_train50.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_34 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_128 (Dense)            (None, 30)                1530      
_________________________________________________________________
dense_129 (Dense)            (None, 4)                 124       
Total params: 1,654
Trainable params: 1,654
Non-trainable params: 0
_________________________________________________________________
Epoch 000/051 | Loss 0.0007 | Win/lose count 4.0/2.0 (2.0)
Epoch 001/051 | Loss 0.0004 | Win/lose count 3.0/8.0 (-5.0)
Epoch 002/051 | Loss 0.0157 | Win/lose count 5.0/5.0 (0.0)
Epoch 003/051 | Loss 0.0083 | Win/lose count 5.5/6.0 (-0.5)
Epoch 004/051 | Loss 0.0106 | Win/lose count 2.0/1.0 (1.0)
Epoch 005/051 | Loss 0.0028 | Win/lose count 4.5/6.0 (-1.5)
Epoch 006/051 | Loss 0.0038 | Win/lose count 3.

I tried several architecture :
- architecture 1 : one 30 fully connected layer
- architecture 2 : one 50 fully connected layer
- architecture 3 : one 50 fully connected layer, one 64 fully connected layer
- architecture 4 : one 50 fully connected layer, one 64 fully connected layer
- architecture 5 : one 50 fully connected layer, one 64 fully connected layer, one 32 fully connected layer
- architecture 6 : same as 5 with dropout

All that networks have been tested with $\epsilon=0.1$ and $lr=0.1$, then tested on 50 games.

| Architecture | Average score   |
|------|------|
|   1  |  2.29  |
|   2  |   2.5  |
|   3  |   2.75 |
|   4  |   0.75 |
|   5  |   0.85 |
|   6  |   0.7  |




Based on the best 3 architectures I then tried to fine tune the learning rate and the $\epsilon$ parameter

In [80]:
test_model(model2,n_epochs=51,id_model=8,epsilon=0.2)
test_model(model3,n_epochs=51,id_model=9,epsilon=0.2)
test_model(model1,n_epochs=51,id_model=10,epsilon=0.2)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_30 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_117 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_118 (Dense)            (None, 4)                 204       
Total params: 2,754
Trainable params: 2,754
Non-trainable params: 0
_________________________________________________________________
Epoch 000/051 | Loss 0.0307 | Win/lose count 4.0/2.0 (2.0)
Epoch 001/051 | Loss 0.0249 | Win/lose count 1.5/1.0 (0.5)
Epoch 002/051 | Loss 0.0249 | Win/lose count 4.0/1.0 (3.0)
Epoch 003/051 | Loss 0.0383 | Win/lose count 1.5/3.0 (-1.5)
Epoch 004/051 | Loss 0.0307 | Win/lose count 3.5/3.0 (0.5)
Epoch 005/051 | Loss 0.0635 | Win/lose count 5.5/4.0 (1.5)
Epoch 006/051 | Loss 0.0080 | Win/lose count 1.5/

Epoch 000/051 | Loss 0.0080 | Win/lose count 3.0/5.0 (-2.0)
Epoch 001/051 | Loss 0.0056 | Win/lose count 5.0/3.0 (2.0)
Epoch 002/051 | Loss 0.0064 | Win/lose count 7.0/4.0 (3.0)
Epoch 003/051 | Loss 0.0114 | Win/lose count 2.0/6.0 (-4.0)
Epoch 004/051 | Loss 0.0060 | Win/lose count 2.5/4.0 (-1.5)
Epoch 005/051 | Loss 0.0069 | Win/lose count 5.5/4.0 (1.5)
Epoch 006/051 | Loss 0.0029 | Win/lose count 2.0/6.0 (-4.0)
Epoch 007/051 | Loss 0.0150 | Win/lose count 4.5/5.0 (-0.5)
Epoch 008/051 | Loss 0.0090 | Win/lose count 4.0/3.0 (1.0)
Epoch 009/051 | Loss 0.0066 | Win/lose count 6.0/4.0 (2.0)
Epoch 010/051 | Loss 0.0153 | Win/lose count 5.0/0 (5.0)
Epoch 011/051 | Loss 0.0189 | Win/lose count 4.5/3.0 (1.5)
Epoch 012/051 | Loss 0.0023 | Win/lose count 1.5/4.0 (-2.5)
Epoch 013/051 | Loss 0.0024 | Win/lose count 3.0/1.0 (2.0)
Epoch 014/051 | Loss 0.0052 | Win/lose count 3.5/4.0 (-0.5)
Epoch 015/051 | Loss 0.0151 | Win/lose count 4.5/1.0 (3.5)
Epoch 016/051 | Loss 0.0072 | Win/lose count 4.5/4.

| Architecture | Epsilon | Average score   |
|------|------|------|
|   1  | 0.1 | 2.29  |
|   2  | 0.1 | 2.5  |
|   3  | 0.1 | **2.75** |
|   1  | 0.2 | -0.91 |
|   2  | 0.2 | 0.67 |
|   3  | 0.2    | 1.03  |


I also tried another optimizer (Adam) but it was not performing well. Changing the learning rate was neither improving the performances.

In [77]:
class DQN_generic(DQN):
    def __init__(self, model,*args, lr=0.1,**kwargs):
        super(DQN_generic, self).__init__( *args,**kwargs)
        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        model.summary()
        self.model = model
test_model(model2,n_epochs=51,id_model=7)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_30 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_117 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_118 (Dense)            (None, 4)                 204       
Total params: 2,754
Trainable params: 2,754
Non-trainable params: 0
_________________________________________________________________
Epoch 000/051 | Loss 0.0316 | Win/lose count 2.0/5.0 (-3.0)
Epoch 001/051 | Loss 0.0158 | Win/lose count 1.0/2.0 (-1.0)
Epoch 002/051 | Loss 0.0250 | Win/lose count 3.0/4.0 (-1.0)
Epoch 003/051 | Loss 0.0502 | Win/lose count 2.5/1.0 (1.5)
Epoch 004/051 | Loss 0.0442 | Win/lose count 1.5/1.0 (0.5)
Epoch 005/051 | Loss 0.0155 | Win/lose count 2.5/4.0 (-1.5)
Epoch 006/051 | Loss 0.0255 | Win/lose count 0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [123]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
      
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(30,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [86]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
#train(agent,env,epochs_train,prefix='cnn_train')
train(agent,env,51,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/051 | Loss 0.0034 | Win/lose count 2.5/7.0 (-4.5)
Epoch 001/051 | Loss 0.0033 | Win/lose count 1.5/1.0 (0.5)
Epoch 002/051 | Loss 0.0037 | Win/lose count 1.5/3.0 (-1.5)
Epoch 003/051 | Loss 0.0690 | Win/lose count 2.5/2.0 (0.5)
Epoch 004/051 | Loss 0.0056 | Win/lose count 6.0/4.0 (2.0)
Epoch 005/051 | Loss 0.0043 | Win/lose count 1.5/4.0 (-2.5)
Epoch 006/051 | Loss 0.0154 | Win/lose count 7.0/5.0 (2.0)
Epoch 007/051 | Loss 0.0039 | Win/lose count 2.5/3.0 (-0.5)
Epoch 008/051 | Loss 0.0030 | Win/lose count 4.0/5.0 (-1.0)
Epoch 009/051 | Loss 0.0034 | Win/lose count 2.5/2.0 (0.5)
Epoch 010/051 | Loss 0.0040 | Win/lose count 9.5/3.0 (6.5)
Epoch 011/051 | Loss 0.0065 | Win/lose count 3.5/2.0 (1.5)
Epoch 012/051 | Loss 0.0058 | Win/lose count 4.5/2.0 (2.5)
Epoch 013/051 | Loss 0.0024 | Win/lose count 2.5/4.0 (-1.5)
Epoch 014/051 | Loss 0.0088 | Win/lose count 5.5/2.0 (3.5)
Epoch 015/051 | Loss 0.0088 | Win/lose count 7.5/3.0 (4.5)
Epoch 016/051 | Loss 0.0032 | Win/lose count 7.5/5

As for FC, I tried different architectures
- architecture 1: 1 hidden Convolution Layer ((50,(2,2)) , relu activations
- architecture 2: 2 hidden Convolution Layer ((50,(2,2) and then (64,(2,2)), relu activations
- architecture 3: 2 hidden Convolution Layer ((50,(2,2),then a MaxPooling layer (2,2) then a  --Convolution layer (64,(2,2)), relu activations
- architecture 4: 1 hidden Convolution Layer ((50,(3,3)) , relu activations
- architecture 5: 1 hidden Convolution Layer ((50,(2,2)) , and then (64,(3,3)) relu activations


In [98]:
cnn_1 = Sequential()
cnn_1.add(Conv2D(50,(2,2),input_shape=(5,5,2,),activation='relu'))
cnn_1.add(Flatten())
cnn_1.add(Dense(4))
test_model(cnn_1,id_model=-1)


###
cnn_2 = Sequential()
cnn_2.add(Conv2D(50,(2,2),input_shape=(5,5,2,),activation='relu'))
cnn_2.add(Conv2D(64,(2,2),activation='relu'))
cnn_2.add(Flatten())
cnn_2.add(Dense(4))
test_model(cnn_2,id_model=-2)

##
cnn_3 = Sequential()
cnn_3.add(Conv2D(50,(2,2),input_shape=(5,5,2,),activation='relu'))
cnn_3.add(MaxPooling2D(2,2))
cnn_3.add(Conv2D(64,(2,2),activation='relu'))
cnn_3.add(Flatten())
cnn_3.add(Dense(4))
test_model(cnn_3,id_model=-3)



        

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 4, 4, 50)          450       
_________________________________________________________________
flatten_53 (Flatten)         (None, 800)               0         
_________________________________________________________________
dense_150 (Dense)            (None, 4)                 3204      
Total params: 3,654
Trainable params: 3,654
Non-trainable params: 0
_________________________________________________________________
Epoch 000/051 | Loss 0.0127 | Win/lose count 3.0/6.0 (-3.0)
Epoch 001/051 | Loss 0.0058 | Win/lose count 3.5/1.0 (2.5)
Epoch 002/051 | Loss 0.0605 | Win/lose count 3.5/2.0 (1.5)
Epoch 003/051 | Loss 0.0017 | Win/lose count 2.0/6.0 (-4.0)
Epoch 004/051 | Loss 0.0041 | Win/lose count 2.0/3.0 (-1.0)
Epoch 005/051 | Loss 0.0015 | Win/lose count 2.0/2.0 (0.0)
Epoch 006/051 | Loss 0.0027 | Win/lose count 2.

Epoch 000/051 | Loss 0.0114 | Win/lose count 5.5/10.0 (-4.5)
Epoch 001/051 | Loss 0.0065 | Win/lose count 2.5/2.0 (0.5)
Epoch 002/051 | Loss 0.0053 | Win/lose count 3.5/4.0 (-0.5)
Epoch 003/051 | Loss 0.0090 | Win/lose count 3.5/5.0 (-1.5)
Epoch 004/051 | Loss 0.0056 | Win/lose count 4.0/4.0 (0.0)
Epoch 005/051 | Loss 0.0047 | Win/lose count 10.5/4.0 (6.5)
Epoch 006/051 | Loss 0.0088 | Win/lose count 6.0/4.0 (2.0)
Epoch 007/051 | Loss 0.0112 | Win/lose count 7.5/3.0 (4.5)
Epoch 019/051 | Loss 0.0031 | Win/lose count 18.0/4.0 (14.0)
Epoch 020/051 | Loss 0.0050 | Win/lose count 1.5/1.0 (0.5)
Epoch 021/051 | Loss 0.0078 | Win/lose count 13.5/6.0 (7.5)
Epoch 022/051 | Loss 0.0052 | Win/lose count 22.0/5.0 (17.0)
Epoch 023/051 | Loss 0.0044 | Win/lose count 16.0/4.0 (12.0)
Epoch 024/051 | Loss 0.0109 | Win/lose count 21.5/3.0 (18.5)
Epoch 025/051 | Loss 0.0054 | Win/lose count 15.5/3.0 (12.5)
Epoch 026/051 | Loss 0.0153 | Win/lose count 17.0/5.0 (12.0)
Epoch 027/051 | Loss 0.0098 | Win/lose

In [99]:
cnn_4 = Sequential()
cnn_4.add(Conv2D(50,(3,3),input_shape=(5,5,2,),activation='relu'))
cnn_4.add(Flatten())
cnn_4.add(Dense(4))
test_model(cnn_4,id_model=-4)


cnn_5 = Sequential()
cnn_5.add(Conv2D(50,(2,2),input_shape=(5,5,2,),activation='relu'))
cnn_5.add(Conv2D(64,(3,3),activation='relu'))
cnn_5.add(Flatten())
cnn_5.add(Dense(4))
test_model(cnn_5,id_model=-5)

    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           (None, 3, 3, 50)          950       
_________________________________________________________________
flatten_56 (Flatten)         (None, 450)               0         
_________________________________________________________________
dense_153 (Dense)            (None, 4)                 1804      
Total params: 2,754
Trainable params: 2,754
Non-trainable params: 0
_________________________________________________________________
Epoch 000/051 | Loss 0.0015 | Win/lose count 4.5/5.0 (-0.5)
Epoch 001/051 | Loss 0.0033 | Win/lose count 2.0/5.0 (-3.0)
Epoch 002/051 | Loss 0.0080 | Win/lose count 1.5/2.0 (-0.5)
Epoch 003/051 | Loss 0.0025 | Win/lose count 2.5/2.0 (0.5)
Epoch 004/051 | Loss 0.0021 | Win/lose count 6.5/3.0 (3.5)
Epoch 005/051 | Loss 0.0006 | Win/lose count 4.5/4.0 (0.5)
Epoch 006/051 | Loss 0.0071 | Win/lose count 3.

Epoch 000/051 | Loss 0.0049 | Win/lose count 2.0/2.0 (0.0)
Epoch 001/051 | Loss 0.0022 | Win/lose count 2.5/2.0 (0.5)
Epoch 002/051 | Loss 0.0185 | Win/lose count 4.5/3.0 (1.5)
Epoch 003/051 | Loss 0.0131 | Win/lose count 3.5/5.0 (-1.5)
Epoch 004/051 | Loss 0.0128 | Win/lose count 0.5/3.0 (-2.5)
Epoch 005/051 | Loss 0.0059 | Win/lose count 2.5/3.0 (-0.5)
Epoch 006/051 | Loss 0.0046 | Win/lose count 11.0/6.0 (5.0)
Epoch 007/051 | Loss 0.0031 | Win/lose count 2.5/3.0 (-0.5)
Epoch 008/051 | Loss 0.0141 | Win/lose count 4.5/5.0 (-0.5)
Epoch 009/051 | Loss 0.0065 | Win/lose count 5.5/5.0 (0.5)
Epoch 010/051 | Loss 0.0110 | Win/lose count 2.5/7.0 (-4.5)
Epoch 011/051 | Loss 0.0081 | Win/lose count 7.5/3.0 (4.5)
Epoch 012/051 | Loss 0.0070 | Win/lose count 3.0/0 (3.0)
Epoch 013/051 | Loss 0.0052 | Win/lose count 5.0/5.0 (0.0)
Epoch 014/051 | Loss 0.0042 | Win/lose count 4.0/5.0 (-1.0)
Epoch 015/051 | Loss 0.0056 | Win/lose count 7.0/2.0 (5.0)
Epoch 016/051 | Loss 0.0041 | Win/lose count 4.5/1

In [101]:
cnn_6 = Sequential()
cnn_6.add(Conv2D(50,(3,3),input_shape=(5,5,2,),activation='relu'))
cnn_6.add(Conv2D(64,(3,3),activation='relu'))
cnn_6.add(Flatten())
cnn_6.add(Dense(4))
test_model(cnn_6,id_model=-6)


cnn_7 = Sequential()
cnn_7.add(Conv2D(50,(3,3),input_shape=(5,5,2,),activation='relu'))
cnn_7.add(Conv2D(64,(2,2),activation='relu'))
cnn_7.add(Flatten())
cnn_7.add(Dense(4))
test_model(cnn_7,id_model=-7)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 3, 3, 50)          950       
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 1, 1, 64)          28864     
_________________________________________________________________
flatten_58 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_155 (Dense)            (None, 4)                 260       
Total params: 30,074
Trainable params: 30,074
Non-trainable params: 0
_________________________________________________________________
Epoch 000/051 | Loss 0.0064 | Win/lose count 1.5/4.0 (-2.5)
Epoch 001/051 | Loss 0.0099 | Win/lose count 5.5/4.0 (1.5)
Epoch 002/051 | Loss 0.0039 | Win/lose count 3.5/2.0 (1.5)
Epoch 003/051 | Loss 0.0129 | Win/lose count 2.5/4.0 (-1.5)
Epoch 004/051 | Loss 0.0031 | Wi

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [106]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)

train(agent_fc, env, 101, prefix='fc_test')
train(agent_cnn, env, 101, prefix='cnn_test')

print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_61 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_158 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_159 (Dense)            (None, 64)                3264      
_________________________________________________________________
dense_160 (Dense)            (None, 4)                 260       
Total params: 6,074
Trainable params: 6,074
Non-trainable params: 0
_________________________________________________________________
Epoch 000/101 | Loss 0.0023 | Win/lose count 4.0/1.0 (3.0)
Epoch 001/101 | Loss 0.0063 | Win/lose count 3.0/4.0 (-1.0)
Epoch 002/101 | Loss 0.0073 | Win/lose count 4.5/7.0 (-2.5)
Epoch 003/101 | Loss 0.0107 | Win/lose count 1.5/2.0 (-0.5)
Epoch 004/101 | Loss 0.0032 | Win

Epoch 023/101 | Loss 0.0034 | Win/lose count 10.5/3.0 (7.5)
Epoch 024/101 | Loss 0.0131 | Win/lose count 11.5/0 (11.5)
Epoch 025/101 | Loss 0.0025 | Win/lose count 9.5/0 (9.5)
Epoch 026/101 | Loss 0.0061 | Win/lose count 15.0/0 (15.0)
Epoch 027/101 | Loss 0.0057 | Win/lose count 7.0/3.0 (4.0)
Epoch 028/101 | Loss 0.0065 | Win/lose count 16.0/4.0 (12.0)
Epoch 029/101 | Loss 0.0028 | Win/lose count 20.0/4.0 (16.0)
Epoch 030/101 | Loss 0.0063 | Win/lose count 11.5/3.0 (8.5)
Epoch 031/101 | Loss 0.0130 | Win/lose count 21.5/0 (21.5)
Epoch 032/101 | Loss 0.0043 | Win/lose count 8.0/1.0 (7.0)
Epoch 033/101 | Loss 0.1586 | Win/lose count 9.0/3.0 (6.0)
Epoch 034/101 | Loss 0.0064 | Win/lose count 22.5/4.0 (18.5)
Epoch 035/101 | Loss 0.0056 | Win/lose count 22.5/5.0 (17.5)
Epoch 036/101 | Loss 0.0093 | Win/lose count 17.0/7.0 (10.0)
Epoch 037/101 | Loss 0.0122 | Win/lose count 12.0/6.0 (6.0)
Epoch 038/101 | Loss 0.0113 | Win/lose count 24.5/3.0 (21.5)
Epoch 039/101 | Loss 0.0232 | Win/lose coun

In [107]:
HTML(display_videos('cnn_test90.mp4'))

In [108]:
HTML(display_videos('fc_test90.mp4'))

As we can see CNN network performs really better than simple Fully connected network. Simple explanation is that the state given to the agent is an image and CNN performs better in image recognition. The difference  increases with the temperature.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [117]:
def update_epsilon(agent):
    if agent.epsilon <= 0.1:
        return
    agent.epsilon -=0.01
    
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action,train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            update_epsilon(agent)
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')


class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
            grid_size = grid_size+4
            self.grid_size = grid_size
            self.max_time = max_time
            self.temperature = temperature

            #board on which one plays
            self.board = np.zeros((grid_size,grid_size))
            self.position = np.zeros((grid_size,grid_size))
            self.malus_position = np.zeros((grid_size,grid_size))

            # coordinate of the cat
            self.x = 0
            self.y = 1

            # self time
            self.t = 0

            self.scale=16

            self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0

        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
            self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]

        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state       

## use those samples of code:
#In train explore:
#state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
#reward = 0
#if train:
 #   reward = -self.malus_position[self.x, self.y]
#self.malus_position[self.x, self.y] = 0.1

#reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
#state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
               #                 self.board.reshape(self.grid_size, self.grid_size,1),
              #          self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [126]:
# Training
epochs_train=101
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore100.mp4'))

Epoch 000/101 | Loss 0.0054 | Win/lose count 5.0/4.0 (1.0)
Epoch 001/101 | Loss 0.0016 | Win/lose count 4.5/5.0 (-0.5)
Epoch 002/101 | Loss 0.1094 | Win/lose count 4.5/4.0 (0.5)
Epoch 003/101 | Loss 0.0116 | Win/lose count 3.5/2.0 (1.5)
Epoch 004/101 | Loss 0.0075 | Win/lose count 6.5/6.0 (0.5)
Epoch 005/101 | Loss 0.0059 | Win/lose count 4.5/2.0 (2.5)
Epoch 006/101 | Loss 0.0924 | Win/lose count 3.5/5.0 (-1.5)
Epoch 007/101 | Loss 0.0046 | Win/lose count 4.5/7.0 (-2.5)
Epoch 008/101 | Loss 0.0028 | Win/lose count 3.0/0 (3.0)
Epoch 009/101 | Loss 0.0015 | Win/lose count 5.0/3.0 (2.0)
Epoch 010/101 | Loss 0.0031 | Win/lose count 1.5/5.0 (-3.5)
Epoch 011/101 | Loss 0.0071 | Win/lose count 6.0/1.0 (5.0)
Epoch 012/101 | Loss 0.0090 | Win/lose count 5.0/3.0 (2.0)
Epoch 013/101 | Loss 0.0135 | Win/lose count 5.0/5.0 (0.0)
Epoch 014/101 | Loss 0.0134 | Win/lose count 2.0/2.0 (0.0)
Epoch 015/101 | Loss 0.0021 | Win/lose count 6.0/3.0 (3.0)
Epoch 016/101 | Loss 0.0051 | Win/lose count 2.5/4.0 (

In [128]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore9.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [153]:
def to_bin(action):
    res = np.zeros(4)
    res[action] = 1
    return res
    
    
def generate_games(agent,number=1000):
    env = Environment(grid_size=size, max_time=T, temperature=0.3)
    states = []
    actions = []
    for i in range(number):
        state = env.reset()
        game_over = False
        while not game_over:
            action = agent.act(state)
            states.append(state)
            actions.append(to_bin(action))
            state,reward,game_over = env.act(action)
    return np.vstack(states),np.vstack(actions)
class DQN_copycat(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        s = s.reshape((1,5,5,self.n_state))
        
        values = self.model.predict(s)
        #print(values)
        return np.argmax(values)
        
    
    def save(self,name_weights='copy_model.h5',name_model='copy_model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='copy_model.h5',name_model='copy_model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "categorical_crossentropy")
        self.model = model
class Copy_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(Copy_CNN, self).__init__(*args,**kwargs)
      
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(30,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "categorical_crossentropy")
        self.model = model
    def train(self, states, actions):
        states=states.reshape((int(states.shape[0]/5),5,5,self.n_state))
        self.model.fit(states,actions,epochs=5)
    

***

In [132]:
states,actions = generate_games(agent_cnn)


In [154]:
copy_agent = Copy_CNN(size, lr=.1, epsilon = 0.1)
copy_agent.train(states,actions)
env = Environment(grid_size=size, max_time=T, temperature=0.3)
#test(copy_agent,env,epochs_test,prefix='cnn_copy')
#HTML(display_videos('cnn_copy9.mp4'))

Epoch 1/5
201000/201000 [==============================] - 25s 125us/step - loss: 5.0176
Epoch 2/5
201000/201000 [==============================] - 20s 101us/step - loss: 5.3174
Epoch 3/5
201000/201000 [==============================] - 19s 94us/step - loss: 5.3174
Epoch 4/5
201000/201000 [==============================] - 19s 96us/step - loss: 5.3174
Epoch 5/5
201000/201000 [==============================] - 22s 111us/step - loss: 5.3174


MemoryError: 

In [ ]:
agent.save('copyCNN.')